In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pylab as plt
import seaborn as sns

In [2]:
data = pd.read_csv('data.csv', encoding='gbk')

In [3]:
## 设置显示所有列
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data.head()

,Unnamed: 0,custid,trade_no,bank_card_no,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,trans_days_interval,regional_mobility,student_feature,repayment_capability,is_high_user,number_of_trans_from_2011,first_transaction_time,historical_trans_amount,historical_trans_day,rank_trad_1_month,trans_amount_3_month,avg_consume_less_12_valid_month,abs,top_trans_count_last_1_month,avg_price_last_12_month,avg_price_top_last_12_valid_month,reg_preference_for_trad,trans_top_time_last_1_month,trans_top_time_last_6_month,consume_top_time_last_1_month,consume_top_time_last_6_month,cross_consume_count_last_1_month,trans_fail_top_count_enum_last_1_month,trans_fail_top_count_enum_last_6_month,trans_fail_top_count_enum_last_12_month,consume_mini_time_last_1_month,max_cumulative_consume_later_1_month,max_consume_count_later_6_month,railway_consume_count_last_12_month,pawns_auctions_trusts_consume_last_1_month,pawns_auctions_trusts_consume_last_6_month,jewelry_consume_count_last_6_month,status,source,first_transaction_day,trans_day_last_12_month,id_name,apply_score,apply_credibility,query_org_count,query_finance_count,query_cash_count,query_sum_count,latest_query_time,latest_one_month_apply,latest_three_month_apply,latest_six_month_apply,loans_score,loans_credibility_behavior,loans_count,loans_settle_count,loans_overdue_count,loans_org_count_behavior,consfin_org_count_behavior,loans_cash_count,latest_one_month_loan,latest_three_month_loan,latest_six_month_loan,history_suc_fee,history_fail_fee,latest_one_month_suc,latest_one_month_fail,loans_long_time,loans_latest_time,loans_credit_limit,loans_credibility_limit,loans_org_count_current,loans_product_count,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
0,5,2791858,2.020000e+31,卡号1,0.01,0.99,0,0.90,0.55,0.313,17.0,27.0,26.0,3.0,NaN,19890,0,30.0,20130817.0,149050,151.0,0.40,34030,7.0,3920,0.15,1020,0.55,一线城市,4.0,19.0,4.0,19.0,1.0,1.0,2.0,2.0,5.0,2170,6.0,0.0,1970,18040,0.0,1,xs,1738.0,85.0,蒋红,583.0,79.0,8.0,2.0,6.0,10.0,2018/4/25,2.0,5.0,8.0,552.0,73.0,37.0,34.0,2.0,10.0,1.0,9.0,1.0,1.0,13.0,37.0,7.0,1.0,0.0,341.0,2018/4/19,2200.0,72.0,9.0,10.0,2900.0,1688.0,1200.0,75.0,1.0,2.0,1200.0,1200.0,12.0,18.0
1,10,534047,2.020000e+31,卡号1,0.02,0.94,2000,1.28,1.00,0.458,19.0,30.0,14.0,4.0,1.0,16970,0,23.0,20160402.0,302910,224.0,0.35,10590,5.0,6950,0.05,1210,0.50,一线城市,13.0,30.0,13.0,30.0,0.0,0.0,3.0,3.0,330.0,2100,9.0,0.0,1820,15680,0.0,0,xs,779.0,84.0,崔向朝,653.0,73.0,7.0,4.0,2.0,8.0,2018/5/3,2.0,6.0,8.0,635.0,76.0,37.0,36.0,0.0,17.0,5.0,12.0,2.0,2.0,8.0,49.0,4.0,2.0,1.0,353.0,2018/5/5,2000.0,74.0,12.0,12.0,3500.0,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.0,2.0
2,12,2849787,2.020000e+31,卡号1,0.04,0.96,0,1.00,1.00,0.114,13.0,68.0,22.0,1.0,NaN,9710,0,9.0,20170617.0,11520,31.0,1.00,5710,5.0,840,0.65,570,0.65,一线城市,0.0,68.0,0.0,68.0,0.0,3.0,6.0,6.0,0.0,0,3.0,0.0,0,0,0.0,1,xs,338.0,95.0,王中云,654.0,76.0,11.0,5.0,5.0,16.0,2018/5/5,5.0,5.0,14.0,633.0,83.0,4.0,2.0,0.0,3.0,1.0,2.0,2.0,2.0,4.0,2.0,2.0,1.0,1.0,157.0,2018/5/1,1500.0,77.0,2.0,2.0,1600.0,1250.0,4200.0,87.0,1.0,1.0,4200.0,4200.0,2.0,6.0
3,13,1809708,2.020000e+31,卡号1,0.00,0.96,2000,0.13,0.57,0.777,22.0,14.0,6.0,3.0,NaN,6210,0,33.0,20130516.0,491130,360.0,0.15,91690,7.0,46850,0.05,1290,0.45,三线城市,6.0,8.0,6.0,8.0,0.0,1.0,8.0,8.0,31700.0,8140,9.0,0.0,2700,27970,0.0,0,xs,1831.0,82.0,何洋洋,595.0,79.0,12.0,7.0,4.0,22.0,2018/5/5,3.0,16.0,17.0,542.0,75.0,85.0,81.0,4.0,22.0,5.0,17.0,2.0,4.0,34.0,91.0,26.0,2.0,0.0,355.0,2018/5/3,1800.0,74.0,17.0,18.0,3200.0,1541.0,16300.0,80.0,5.0,5.0,30000.0,12180.0,2.0,4.0
4,14,2499829,2.020000e+31,卡号1,0.01,0.99,0,0.46,1.00,0.175,13.0,66.0,42.0,1.0,NaN,11150,0,12.0,20170312.0,61470,63.0,0.65,9770,6.0,760,1.00,1110,0.50,一线城市,0.0,66.0,0.0,66.0,0.0,3.0,3.0,3.0,0

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4754 entries, 0 to 4753
Data columns (total 90 columns):
Unnamed: 0                                    4754 non-null int64
custid                                        4754 non-null int64
trade_no                                      4754 non-null float64
bank_card_no                                  4754 non-null object
low_volume_percent                            4752 non-null float64
middle_volume_percent                         4752 non-null float64
take_amount_in_later_12_month_highest         4754 non-null int64
trans_amount_increase_rate_lately             4751 non-null float64
trans_activity_month                          4752 non-null float64
trans_activity_day                            4752 non-null float64
transd_mcc                                    4752 non-null float64
trans_days_interval_filter                    4746 non-null float64
trans_days_interval                           4752 non-null float64
regional_mobilit

In [5]:
# 构造出特征值
## X = data.drop(['status'], axis=1)
X = data
# data['status']

In [6]:
# 构造出目标值  ## 在最后再构造，否则可能会分出来的labels会错过一些基本的数据处理
## labels = data['status']

In [7]:
## labels.tail()

## 1.1 初步查看数据类型

In [8]:
# 通过info查看每列的数据是什么类型，也可以找出类别型的数据（类别型的为object类型）
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4754 entries, 0 to 4753
Data columns (total 90 columns):
Unnamed: 0                                    4754 non-null int64
custid                                        4754 non-null int64
trade_no                                      4754 non-null float64
bank_card_no                                  4754 non-null object
low_volume_percent                            4752 non-null float64
middle_volume_percent                         4752 non-null float64
take_amount_in_later_12_month_highest         4754 non-null int64
trans_amount_increase_rate_lately             4751 non-null float64
trans_activity_month                          4752 non-null float64
trans_activity_day                            4752 non-null float64
transd_mcc                                    4752 non-null float64
trans_days_interval_filter                    4746 non-null float64
trans_days_interval                           4752 non-null float64
regional_mobilit

In [9]:
# 发现类别型（object）的列名有： X['bank_card_no','reg_preference_for_trad','source','id_name','latest_query_time','loans_latest_time']
for objects in ['bank_card_no','reg_preference_for_trad','source']:
    print(Counter(X[objects]))
    print('')

Counter({'卡号1': 4754})

Counter({'一线城市': 3403, '三线城市': 1064, '境外': 150, '二线城市': 131, '其他城市': 4, nan: 2})

Counter({'xs': 4754})



In [10]:
# 将城市进行数值化
map_dict = {'一线城市': 1, '二线城市':2, '三线城市': 3, '境外': 4, '其他城市': 5,}
X['reg_preference_for_trad'] = X['reg_preference_for_trad'].map(map_dict)
X['reg_preference_for_trad'].head(10)

0    1.0
1    1.0
2    1.0
3    3.0
4    1.0
5    3.0
6    1.0
7    1.0
8    3.0
9    1.0
Name: reg_preference_for_trad, dtype: float64

In [11]:
# 将日期处理成datatime格式
X['latest_query_time'] = pd.to_datetime(X['latest_query_time'])
X['loans_latest_time'] = pd.to_datetime(X['loans_latest_time'])
X['loans_latest_time'].head()

0   2018-04-19
1   2018-05-05
2   2018-05-01
3   2018-05-03
4   2018-01-07
Name: loans_latest_time, dtype: datetime64[ns]

In [12]:
X.select_dtypes(include=['object']).head(5)

,bank_card_no,source,id_name
0,卡号1,xs,蒋红
1,卡号1,xs,崔向朝
2,卡号1,xs,王中云
3,卡号1,xs,何洋洋
4,卡号1,xs,赵洋


### 删除一些只有一类或ID型的列

In [13]:
X = X.drop(['bank_card_no','source','id_name','Unnamed: 0'], axis=1)

In [14]:
X = X.drop(['loans_latest_time', 'latest_query_time'], axis=1)

### 缺失值处理

In [15]:
X.isnull().sum()

custid                                           0
trade_no                                         0
low_volume_percent                               2
middle_volume_percent                            2
take_amount_in_later_12_month_highest            0
trans_amount_increase_rate_lately                3
trans_activity_month                             2
trans_activity_day                               2
transd_mcc                                       2
trans_days_interval_filter                       8
trans_days_interval                              2
regional_mobility                                2
student_feature                               2998
repayment_capability                             0
is_high_user                                     0
number_of_trans_from_2011                        2
first_transaction_time                           2
historical_trans_amount                          0
historical_trans_day                             2
rank_trad_1_month              

In [16]:
X = X.drop(['custid', 'student_feature'], axis=1)

In [17]:
# 删除缺失值的数据
print('before:' + str(X.shape))
X.dropna(inplace=True)
print(X.shape)

before:(4754, 82)
(3983, 82)


In [18]:
# 删除方差没有变化的数据
X.drop(X.columns[X.std()==0], axis=1, inplace=True)
print(X.shape)

(3983, 82)


In [19]:
np.round(X.corr(method='pearson'), 2)
#热力图研究相关性
# plt.figure(figsize=(62,54))
# corr=X.corr()
# sns.heatmap(corr,fmt='.1g',annot=True,cbar=False)

## 由于能力问题，什么也没看出来 ~~~~(>_<)~~~~  ## 

,trade_no,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,trans_days_interval,regional_mobility,repayment_capability,is_high_user,number_of_trans_from_2011,first_transaction_time,historical_trans_amount,historical_trans_day,rank_trad_1_month,trans_amount_3_month,avg_consume_less_12_valid_month,abs,top_trans_count_last_1_month,avg_price_last_12_month,avg_price_top_last_12_valid_month,reg_preference_for_trad,trans_top_time_last_1_month,trans_top_time_last_6_month,consume_top_time_last_1_month,consume_top_time_last_6_month,cross_consume_count_last_1_month,trans_fail_top_count_enum_last_1_month,trans_fail_top_count_enum_last_6_month,trans_fail_top_count_enum_last_12_month,consume_mini_time_last_1_month,max_cumulative_consume_later_1_month,max_consume_count_later_6_month,railway_consume_count_last_12_month,pawns_auctions_trusts_consume_last_1_month,pawns_auctions_trusts_consume_last_6_month,jewelry_consume_count_last_6_month,status,first_transaction_day,trans_day_last_12_month,apply_score,apply_credibility,query_org_count,query_finance_count,query_cash_count,query_sum_count,latest_one_month_apply,latest_three_month_apply,latest_six_month_apply,loans_score,loans_credibility_behavior,loans_count,loans_settle_count,loans_overdue_count,loans_org_count_behavior,consfin_org_count_behavior,loans_cash_count,latest_one_month_loan,latest_three_month_loan,latest_six_month_loan,history_suc_fee,history_fail_fee,latest_one_month_suc,latest_one_month_fail,loans_long_time,loans_credit_limit,loans_credibility_limit,loans_org_count_current,loans_product_count,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
trade_no,1.0,0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00
low_volume_percent,0.0,1.00,-0.02,-0.08,-0.01,0.02,-0.08,-0.02,0.02,0.05,-0.03,-0.05,0.02,-0.03,0.04,-0.14,-0.07,0.05,-0.07,0.01,-0.10,-0.03,-0.37,0.60,0.03,-0.00,0.02,-0.01,0.02,0.06,0.02,-0.02,-0.01,-0.01,-0.05,0.11,0.02,-0.07,-0.17,0.02,0.06,-0.04,0.04,0.03,0.01,-0.08,-0.08,-0.06,-0.08,-0.05,-0.07,-0.07,0.03,0.05,-0.14,-0.16,-0.02,-0.16,-0.10,-0.16,-0.05,-0.05,-0.10,-0.10,-0.00,-0.02,0.03,-0.10,-0.20,-0.11,-0.16,-0.16,-0.21,-0.19,-0.09,-0.03,-0.10,-0.09,-0.12,-0.11,0.07,0.04
middle_volume_percent,-0.0,-0.02,1.00,-0.16,-0.05,-0.03,-0.09,-0.16,0.03,0.06,-0.01,-0.18,-0.01,-0.11,0.07,-0.30,-0.15,0.18,-0.21,-0.18,-0.26,0.09,-0.57,0.41,-0.13,-0.01,0.04,-0.02,0.05,-0.08,0.03,0.09,0.09,0.03,-0.26,-0.02,-0.00,-0.23,-0.39,-0.06,0.04,-0.07,0.06,-0.02,0.00,0.06,0.04,0.09,0.04,-0.01,-0.01,0.01,-0.04,-0.03,0.10,0.09,0.06,0.09,-0.01,0.13,-0.02,-0.00,0.08,0.04,0.06,-0.03,-0.03,0.04,-0.11,0.07,0.13,0.13,-0.17,-0.14,-0.16,-0.00,-0.01,-0.02,-0.18,-0.19,0.02,0.02
take_amount_in_later_12_month_highest,0.0,-0.08,-0.16,1.00,0.00,0.06,0.10,0.18,-0.06,-0.04,0.00,0.05,0.05,0.05,-0.00,0.10,0.10,-0.08,0.07,0.02,0.07,-0.06,0.14,-0.13,0.07,0.02,-0.07,0.02,-0.06,0.04,-0.00,-0.01,0.01,-0.03,0.05,0.00,0.02,0.09,0.16,0.02,-0.01,0.00,-0.05,-0.01,-0.01,-0.00,-0.01,-0.00,-0.01,0.02,0.01,-0.01,-0.01,-0.02,0.03,0.04,-0.02,0.01,0.01,0.01,0.01,0.01,0.03,0.03,-0.02,0.02,-0.00,0.01,0.05,-0.03,0.01,0.01,0.07,0.03,0.04,0.01,0.01,0.02,0.05,0.04,-0.00,-0.00
trans_amount_increase_rate_lately,0.0,-0.01,-0.05,0.00,1.00,0.01,-0.01,-0.01,0.02,-0.01,0.00,0.01,-0.00,-0.01,0.01,0.01,-0.01,0.03,-0.00,0.02,0.00,0.04,0.03,-0.04,0.03,-0.03,0.02,-0.03,0.02,-0.00,-0.0

## 1.2 将数据划分为训练集和测试集

In [21]:
labels = X['status']
X = X.drop(['status'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(X, labels, random_state=2018, test_size=0.3)

In [22]:
Counter(data['reg_preference_for_trad'])

Counter({1.0: 3403, 3.0: 1064, 4.0: 150, 2.0: 131, 5.0: 4, nan: 1, nan: 1})

In [23]:
data['reg_preference_for_trad'].value_counts()

1.0    3403
3.0    1064
4.0     150
2.0     131
5.0       4
Name: reg_preference_for_trad, dtype: int64

In [25]:
X['reg_preference_for_trad'].value_counts()

1.0    2942
3.0     866
2.0      92
4.0      79
5.0       4
Name: reg_preference_for_trad, dtype: int64

####  1.2.1 使用决策树进行建模

In [26]:
from sklearn.tree import DecisionTreeClassifier
dec = DecisionTreeClassifier()
dec.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [27]:
# 预测精确率
print('预测精确率： ' + str(dec.score(x_test, y_test)))

预测精确率： 0.7062761506276151


####  1.2.2 使用KNN建立模型

In [28]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [29]:
print('预测精确率：' + str(knn.score(x_test, y_test)))

预测精确率：0.703765690376569


####  1.2.3 使用随机森林建立模型

In [30]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train, y_train)

c:\users\kljdgf\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [31]:
print('预测精确率：' + str(rf.score(x_test, y_test)))

预测精确率：0.7765690376569038


In [33]:
# SVC
from sklearn.svm import SVC

svc = SVC(kernel="rbf", C=1)
svc.fit(x_train, y_train)


c:\users\kljdgf\appdata\local\programs\python\python36\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [34]:
print('预测精确率： ' + str(svc.score(x_test, y_test)))

预测精确率： 0.7698744769874477
